# VacationPy
----

# Analysis

1. There is a strong negative corelation between the latitude and the max temperature. The higher the latitude is, the lower the max temperature gets.

2. There is nearly no correlation between wind speed and latitude in both hemisheres.

3. There is weak negative correlation between humidity and the latitude in the southern hemisphere. The higher the latitude, the more likely the humidity level will decrease.

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key
from pprint import pprint

# Import API key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [66]:
df=pd.read_csv('output.csv')
df

,city,country,date,lat,lon,max_temp,humidity,cloudiness,wind_speed
0,São Gabriel da Cachoeira,BR,1631344798,-0.1303,-67.0892,22.12,99,100,0.79
1,Rikitea,PF,1631344799,-23.1203,-134.9692,23.34,74,100,5.36
2,Ilulissat,GL,1631344800,69.2167,-51.1000,-4.99,39,0,1.54
3,Ushuaia,AR,1631344801,-54.8000,-68.3000,2.81,87,20,1.54
4,Bredasdorp,ZA,1631344803,-34.5322,20.0403,12.62,94,100,0.51
...,...,...,...,...,...,...,...,...,...
537,Ugoofaaru,MV,1631345278,5.6667,73.0000,28.15,71,74,4.85
538,Bien Hoa,VN,1631345279,10.9500,106.8167,31.99,78,100,6.42
539,Seoul,KR,1631345006,37.5683,126.9778,29.66,52,20,5.14
540,Pemangkat,ID,1631345281,1.1667,108.9667,30.53,62,99,2.69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:

figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [29]:
location=df[["lat", "lon"]]
heat_layer = gmaps.heatmap_layer(location, weights=df['humidity'], 
                                 dissipating=False, max_intensity=max(df['humidity']),
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
# df.info()
# For part II a dataframe is created that contains the following:
# ✓ Ten or less rows.
# ✓ City
# ✓ Country
# ✓ Latitude
# ✓ Longitude 
# ✓ Hotel Name

df=df.dropna()
df.info()

new_df=df.loc[(df['lat']>-80) & (df['lat']<60)& (df['wind_speed']>2.8)
              &(df['cloudiness']<70)].sample(10)
new_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540 entries, 0 to 541
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        540 non-null    object 
 1   country     540 non-null    object 
 2   date        540 non-null    int64  
 3   lat         540 non-null    float64
 4   lon         540 non-null    float64
 5   max_temp    540 non-null    float64
 6   humidity    540 non-null    int64  
 7   cloudiness  540 non-null    int64  
 8   wind_speed  540 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 42.2+ KB


,city,country,date,lat,lon,max_temp,humidity,cloudiness,wind_speed
230,Manacor,ES,1631345000,39.5696,3.2096,25.28,63,22,3.25
150,Gucheng,CN,1631344927,35.9115,111.3192,33.34,27,0,4.56
31,Malbork,PL,1631344827,54.0359,19.0266,21.05,58,11,4.71
266,Mopipi,BW,1631345033,-21.1833,24.8833,18.80,40,0,7.96
465,Bathsheba,BB,1631345212,13.2167,-59.5167,28.34,88,40,6.17
534,Preobrazheniye,RU,1631345275,42.9019,133.9064,22.16,68,11,3.75
427,Haining,CN,1631345180,30.5363,120.6864,27.01,81,8,6.22
453,Big Spring,US,1631345202,32.2504,-101.4787,25.25,41,1,5.56
172,Itarema,BR,1631344947,-2.9248,-39.9167,25.44,87,31,6.38
344,Kavieng,PG,1631344991,-2.5744,150.7967,28.98,70,6,5.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [77]:
hotel_df=new_df
hotel_df['Hotel Name']=''

# hotel_df
base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {
    'keyword':'hotel',
    "radius": 5000,
    "type": 'hotel',
    "key":  g_key
}
for index,row in hotel_df.iterrows():
    city=row['city']
    lat=row['lat']
    lon=row['lon']
    params['location']=f'{lat},{lon}'
    response=requests.get(base_url,params=params).json()
    try:
        hotel_name=response['results'][0]['name']
        hotel_df.loc[index,'Hotel Name']=hotel_name
        print(f'Retrieving Results for Index {index}: {city}')
    except:
        print('Error, no result found')


Retrieving Results for Index 230: Manacor
Error, no result found
Retrieving Results for Index 31: Malbork
Error, no result found
Retrieving Results for Index 465: Bathsheba
Retrieving Results for Index 534: Preobrazheniye
Retrieving Results for Index 427: Haining
Retrieving Results for Index 453: Big Spring
Retrieving Results for Index 172: Itarema
Retrieving Results for Index 344: Kavieng


In [78]:
# hotel_df

,city,country,date,lat,lon,max_temp,humidity,cloudiness,wind_speed,Hotel Name
230,Manacor,ES,1631345000,39.5696,3.2096,25.28,63,22,3.25,Hotel Can Guixa
150,Gucheng,CN,1631344927,35.9115,111.3192,33.34,27,0,4.56,
31,Malbork,PL,1631344827,54.0359,19.0266,21.05,58,11,4.71,Edmar - Hotel Rooms and Restaurant
266,Mopipi,BW,1631345033,-21.1833,24.8833,18.80,40,0,7.96,
465,Bathsheba,BB,1631345212,13.2167,-59.5167,28.34,88,40,6.17,Atlantis Hotel
534,Preobrazheniye,RU,1631345275,42.9019,133.9064,22.16,68,11,3.75,Campsite Orlan
427,Haining,CN,1631345180,30.5363,120.6864,27.01,81,8,6.22,"Langham Place, Haining"
453,Big Spring,US,1631345202,32.2504,-101.4787,25.25,41,1,5.56,La Quinta Inn & Suites by Wyndham Big Spring
172,Itarema,BR,1631344947,-2.9248,-39.9167,25.44,87,31,6.38,Vila Ybytu Eco Resort
344,Kavieng,PG,1631344991,-2.5744,150.7967,28.98,70,6,5.87,Nusa Island Retreat


In [79]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [80]:
# Add marker layer ontop of heat map

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))